# Decompositions and least squares


We now look at several decompositions (or factorisations) 
of a matrix into products of structured matrices, and their use in solving least squares problems and linear systems.
For a square or rectangular matrix $A ∈ ℝ^{m × n}$ with more rows than columns ($m ≥ n$), we consider:
1. The _QR decomposition_
$$
A = Q R = \underbrace{\begin{bmatrix} 𝐪_1 | \cdots | 𝐪_m \end{bmatrix}}_{m × m} \underbrace{\begin{bmatrix} × & \cdots & × \\ & ⋱ & ⋮ \\ && × \\ &&0 \\ &&⋮ \\ && 0 \end{bmatrix}}_{m × n}
$$
where $Q$ is orthogonal ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) and $R$ is _right triangular_, which means it 
is only nonzero on or to the right of the diagonal.

2. The _reduced QR decomposition_
$$
A = Q̂ R̂ = \underbrace{\begin{bmatrix} 𝐪_1 | \cdots | 𝐪_n \end{bmatrix}}_{m × n} \underbrace{\begin{bmatrix} × & \cdots & × \\ & ⋱ & ⋮ \\ && ×  \end{bmatrix}}_{n × n}
$$
where $Q$ has orthogonal columns ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) and $R̂$ is upper triangular.

For a square matrix we consider the _PLU decomposition_:
$$
A = P^⊤ LU
$$
where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular.

Finally, for a square, _symmetric positive definite_ ($𝐱^⊤ A 𝐱 > 0$ for all $𝐱 ∈ ℝ^n$) 
matrix we consider the _Cholesky decomposition_:
$$
A = L L^⊤
$$

The importance of these decomposition for square matrices is that their component pieces are easy to invert on a computer:
$$
\begin{align*}
A = P^⊤ LU &\Rightarrow\qquad A^{-1}𝐛 = U^{-1} L^{-1} P 𝐛 \\
A = QR &\Rightarrow\qquad A^{-1}𝐛 = R^{-1} Q^\top 𝐛 \\
A = L L^⊤ &\Rightarrow\qquad A^{-1}𝐛 = L^{-⊤} L^{-1} 𝐛
\end{align*}
$$
and we saw last lecture that triangular and orthogonal matrices are easy to invert when applied to a vector $𝐛$,
e.g., using forward/back-substitution.
For rectangular matrices we will see that they lead to efficient solutions to the _least squares problem_: find
$𝐱$ that minimizes the 2-norm
$$
\| A 𝐱 - 𝐛 \|.
$$

In this lecture we discuss the followng:

1. QR and least squares: We discuss the QR decomposition and its usage in solving least squares problems.
2. Reduced QR and Gram–Schmidt: We discuss computation of the Reduced QR decomposition using Gram–Schmidt.
3. Householder reflections and QR: We discuss comuting the  QR decomposition using Householder reflections.
2. PLU decomposition: we discuss how the LU decomposition can be computed using Gaussian elimination, and the computation of
the PLU decomposition via Gaussian elimination with pivoting.
3. Cholesky decomposition: we introduce symmetric positive definite matrices and show that their LU decomposition can be re-interpreted
as a Cholesky decomposition.
4. Timings: we see the relative trade-off in speed between the different decompositions.

In [1]:
using LinearAlgebra, Plots, BenchmarkTools

## 1. QR and least squares

Here we consider rectangular matrices with more rows than columns.
A QR decomposition decomposes a matrix into an orthogonal matrix $Q$ times a right triangular matrix $R$. 
Note the QR decomposition contains within it the reduced QR decomposition:
$$
A = QR = \begin{bmatrix} Q̂ | 𝐪_{n+1} | ⋯ | 𝐪_m \end{bmatrix} \begin{bmatrix} R̂ \\  𝟎_{m-n × n} \end{bmatrix} = Q̂ R̂.
$$


We can use it to solve a least squares problem using the norm-preserving property (see PS3) of orthogonal matrices:
$$
\| A 𝐱 - 𝐛 \| = \| Q R 𝐱 - 𝐛 \| = \| R 𝐱 - Q^⊤ 𝐛 \| = \left \| 
\begin{bmatrix} R̂ \\ 𝟎_{m-n × n} \end{bmatrix} 𝐱 - \begin{bmatrix} Q̂^⊤ \\ 𝐪_{n+1}^⊤ \\ ⋮ \\ 𝐪_m^⊤ \end{bmatrix}     𝐛 \right \|
$$
Now note that the rows $k > n$ are independent of $𝐱$ and are a fixed contribution. Thus to minimise this norm it suffices to
drop them and minimise:
$$
\| R̂ 𝐱 - Q̂^⊤ 𝐛 \|
$$
This norm is minimisable if it is attained. Provided the column rank of $A$ is full, $R̂$ will be invertible (Exercise: why is this?).
Thus we have the solution
$$
𝐱 = R̂^{-1} Q̂^⊤ 𝐛
$$

**Example (quadratic fit)** Suppose we want to fit noisy data by a quadratic
$$
p(x) = p₀ + p₁ x + p₂ x^2
$$
That is, we want to choose $p₀,p₁,p₂$ at data samples $x_1, \ldots, x_m$ so that the following is true:
$$
p₀ + p₁ x_k + p₂ x_k^2 ≈ f_k
$$
where $f_k$ are given by data. We can reinterpret this as a least squares problem: minimise the norm
$$
\left\| \begin{bmatrix} 1 & x_1 & x_1^2 \\ ⋮ & ⋮ & ⋮ \\ 1 & x_m & x_m^2 \end{bmatrix}
\begin{bmatrix} p₀ \\ p₁ \\ p₂ \end{bmatrix} - \begin{bmatrix} f_1 \\ ⋮ \\ f_m \end{bmatrix} \right \|
$$
We can solve this using the QR decomposition:

In [2]:
m,n = 100,3

x = range(0,1; length=m) # 100 points
f = 2 .+ x .+ 2x.^2 .+ 0.1 .* randn.() # Noisy quadratic

A = x .^ (0:2)'  # 100 x 3 matrix, equivalent to [ones(m) x x.^2]
Q,R̂ = qr(A)
Q̂ = Q[:,1:n] # Q represents full orthogonal matrix so we take first 3 columns

p₀,p₁,p₂ = R̂ \ Q̂'f

3-element Vector{Float64}:
 2.0020823730946438
 1.0164297831971307
 1.9480305651560517

We can visualise the fit:

In [3]:
p = x -> p₀ + p₁*x + p₂*x^2

scatter(x, f; label="samples", legend=:bottomright)
plot!(x, p.(x); label="quadratic")

Note that `\` with a rectangular system does least squares by default:

In [4]:
A \ f

3-element Vector{Float64}:
 2.002082373094645
 1.0164297831971287
 1.9480305651560528

## 2. Reduced QR and Gram–Schmidt


How do we compute the QR decomposition? We begin with a method
you may have seen before in another guise. Write
$$
A = \begin{bmatrix} 𝐚_1 | \dots | 𝐚_n \end{bmatrix}
$$
where $𝐚_k \in  ℝ^m$ and assume they are linearly independent ($A$ has full column rank).
Note that the column span of the first $j$ columns of $A$
will be the same as the first $j$ columns of $Q̂$, as
$R̂$ must be non-singular:
$$
\hbox{span}(𝐚_1,\ldots,𝐚_j) = \hbox{span}(𝐪_1,\ldots,𝐪_j)
$$
In other words: the columns of $Q̂$ are an orthogonal basis
of the column span of $A$.
To see this note that since `R̂` is triangular we have
$$
\begin{bmatrix} 𝐚_1 | \dots | 𝐚_j \end{bmatrix} = \begin{bmatrix} 𝐪_1 | \dots | 𝐪_j \end{bmatrix}  R̂[1:j,1:j]
$$
for all $j$. That is, if $𝐯 ∈ \hbox{span}(𝐚_1,\ldots,𝐚_j)$ we have for $𝐜 ∈ ℝ^j$
$$
𝐯 = \begin{bmatrix} 𝐚_1 | \dots | 𝐚_j \end{bmatrix} 𝐜 = \begin{bmatrix} 𝐪_1 | \dots | 𝐪_j \end{bmatrix}  R̂[1:j,1:j] 𝐜 ∈ \hbox{span}(𝐪_1,\ldots,𝐪_j)
$$
 while if $𝐰 ∈ \hbox{span}(𝐪_1,\ldots,𝐪_j)$ we have for $𝐝 ∈ ℝ^j$
$$
𝐰 = \begin{bmatrix} 𝐪_1 | \dots | 𝐪_j \end{bmatrix} 𝐝  =  \begin{bmatrix} 𝐚_1 | \dots | 𝐚_j \end{bmatrix} R̂[1:j,1:j]^{-1} 𝐝 ∈  \hbox{span}(𝐚_1,\ldots,𝐚_j).
$$

 
It is possible to find an orthogonal basis using the _Gram–Schmidt algorithm_,
We construct it via induction:
assume that
$$
\hbox{span}(𝐚_1,\ldots,𝐚_{j-1}) = \hbox{span}(𝐪_1,\ldots,𝐪_{j-1})
$$
where $𝐪_1,\ldots,𝐪_{j-1}$ are orthogonal:
$$
𝐪_k^\top 𝐪_ℓ = δ_{kℓ} = \begin{cases} 1 & k = ℓ \\
                                            0 & \hbox{otherwise} \end{cases}.
$$
for $k,ℓ < j$.
Define
$$
𝐯_j := 𝐚_j - \sum_{k=1}^{j-1} \underbrace{𝐪_k^\top 𝐚_j}_{r_{kj}} 𝐪_k
$$
so that for $k < j$
$$
𝐪_k^\top 𝐯_j = 𝐪_k^\top 𝐚_j - \sum_{k=1}^{j-1} \underbrace{𝐪_k^\top 𝐚_j}_{r_{kj}} 𝐪_k^\top 𝐪_k = 0.
$$
Then we define
$$
𝐪_j := {𝐯_j \over \|𝐯_j\|}.
$$
which sastisfies $𝐪_k^\top 𝐪_j =δ_{kj}$ for $k \leq j$.

We now reinterpret this construction as a reduced QR decomposition.
Define
$
r_{jj} := {\|𝐯_j\|}
$
Then rearrange the definition we have
$$
𝐚_j = \begin{bmatrix} 𝐪_1|\cdots|𝐪_j \end{bmatrix} \begin{bmatrix} r_{1j} \\ ⋮ \\ r_{jj} \end{bmatrix}
$$
Thus
$$
\begin{bmatrix} 𝐚_1|\cdots|𝐚_j \end{bmatrix} 
\begin{bmatrix} r_{11} & \cdots & r_{1j} \\ & ⋱ & ⋮ \\ && r_{jj} \end{bmatrix}
$$
That is, we are computing the reduced QR decomposition column-by-column. 
Running this algorithm to $j = n$ completes the decomposition.

### Gram–Schmidt in action

We are going to compute the reduced QR of a random matrix

In [5]:
m,n = 5,4
A = randn(m,n)
Q,R̂ = qr(A)
Q̂ = Q[:,1:n]

5×4 Matrix{Float64}:
 -0.640643   -0.0823157  -0.22609     0.728861
 -0.0996754  -0.285725    0.575686    0.0805714
  0.490733   -0.793914   -0.166541    0.285984
 -0.580068   -0.530002   -0.0540479  -0.591485
 -0.0484121  -0.0195648   0.766036    0.175009

The first column of `Q̂` is indeed a normalised first column of `A`:

In [6]:
R = zeros(n,n)
Q = zeros(m,n)
R[1,1] = norm(A[:,1])
Q[:,1] = A[:,1]/R[1,1]

5-element Vector{Float64}:
 -0.6406428105326175
 -0.09967542686075603
  0.49073256021653416
 -0.5800684602848314
 -0.04841212987574009

We now determine the next entries as

In [7]:
R[1,2] = Q[:,1]'A[:,2]
v = A[:,2] - Q[:,1]*R[1,2]
R[2,2] = norm(v)
Q[:,2] = v/R[2,2]

5-element Vector{Float64}:
 -0.08231573808619484
 -0.2857251402909879
 -0.7939143184765172
 -0.5300023946604269
 -0.01956476529889328

And the third column is then:

In [8]:
R[1,3] = Q[:,1]'A[:,3]
R[2,3] = Q[:,2]'A[:,3]
v = A[:,3] - Q[:,1:2]*R[1:2,3]
R[3,3] = norm(v)
Q[:,3] = v/R[3,3]

5-element Vector{Float64}:
  0.22608968882009056
 -0.575686495796397
  0.16654081939416812
  0.05404794982633636
 -0.7660362170041234

(Note the signs may not necessarily match.)

We can clean this up as a simple algorithm:

In [9]:
function gramschmidt(A)
    m,n = size(A)
    m ≥ n || error("Not supported")
    R = zeros(n,n)
    Q = zeros(m,n)
    for j = 1:n
        for k = 1:j-1
            R[k,j] = Q[:,k]'*A[:,j]
        end
        v = A[:,j] - Q[:,1:j-1]*R[1:j-1,j]
        R[j,j] = norm(v)
        Q[:,j] = v/R[j,j]
    end
    Q,R
end

Q,R = gramschmidt(A)
norm(A - Q*R)

2.4873781269328544e-16

### Complexity and stability

We see within the `for j = 1:n` loop that we have $O(mj)$ operations. Thus the 
total complexity is $O(m n^2)$ operations.


Unfortunately, the Gram–Schmidt algorithm is _unstable_: the rounding errors when implemented in floating point
accumulate in a way that we lose orthogonality:

In [10]:
A = randn(300,300)
Q,R = gramschmidt(A)
norm(Q'Q-I)

3.5531891494246798e-12

## 3. Householder reflections and QR

As an alternative, we will consider using Householder reflections to introduce zeros below
the diagonal.
Thus, if Gram–Schmidt is a process of _triangular orthogonalisation_ (using triangular matrices
to orthogonalise), Householder reflections is a process of _orthogonal triangularisation_ 
(using orthogonal matrices to triangularise).

Consider multiplication by the Householder reflection corresponding to the first column,
that is, for
$$
Q_1 := Q_{𝐚_1}^{\rm H},
$$
consider
$$
Q_1 A = \begin{bmatrix} \times & \times & \cdots & \times \\
& \times & \cdots & \times \\
                    & ⋮ & ⋱ & ⋮ \\
                    & \times & \cdots & \times \end{bmatrix} = 
\begin{bmatrix} r_{11} & r_{12} & \cdots & r_{1n} \\ 
& 𝐚_2^1 & \cdots & 𝐚_n^1   \end{bmatrix}
$$
where 
$$
r_{1j} :=  (Q_1 𝐚_j)[1] \qquad \hbox{and} \qquad 𝐚_j^1 := (Q_1 𝐚_j)[2:m],
$$
noting $r_{11} = -\hbox{sign}(a_{11})\|𝐚_1\|$ and all entries of $𝐚_1^1$ are zero (thus not included).
That is, we have made the first column triangular.

But now consider
$$
Q_2 := \begin{bmatrix} 1  \\ & Q_{𝐚_2^1}^{\rm H} \end{bmatrix} = Q_{\begin{bmatrix} 0 \\ 𝐚_2^1 \end{bmatrix}}^H
$$
so that
$$
Q_2 Q_1A = \begin{bmatrix} \times & \times & \times & \cdots & \times \\
& \times & \times  & \cdots & \times \\
                    && ⋮ & ⋱ & ⋮ \\
                    && \times & \cdots & \times \end{bmatrix}  =  \begin{bmatrix} r_{11} & r_{12} & r_{13} & \cdots & r_{1n} \\ 
    & r_{22} & r_{23} & \cdots & r_{2n} \\
&& 𝐚_3^2 & \cdots & 𝐚_n^2   \end{bmatrix}
$$
where 
$$
r_{2j} :=  (Q_2 𝐚_j^1)[1] \qquad \hbox{and} \qquad 𝐚_j^2 := (Q_2 𝐚_j^1)[2:m-1]
$$
Thus the first two columns are triangular. 

The inductive construction is thus clear. If we define $𝐚_j^0 := 𝐚_j$ we
have the construction
$$
\begin{align*}
Q_j &:= \begin{bmatrix} I_{j-1 \times j-1}  \\ & Q_{𝐚_j^j}^{±,\rm H} \end{bmatrix} \\
𝐚_j^k &:= (Q_k 𝐚_j^{k-1})[2:m-k+1] \\
r_{kj} &:= (Q_k 𝐚_j^{k-1})[1]
\end{align*}
$$
Then
$$
Q_n \cdots Q_1 A = \underbrace{\begin{bmatrix} 
r_{11} & \cdots & r_{1n} \\ & ⋱ & ⋮\\
                                        && r_{nn} \\&& 0 \\ && ⋮ \\ && 0 \end{bmatrix}}_R
$$
i.e.
$$
A = \underbrace{Q_n \cdots Q_1}_Q R.
$$

The implementation is cleaner. We do a naive implementation here:

In [11]:
function householderreflection(x)
    y = copy(x)
    # we cannot use sign(x[1]) in case x[1] == 0
    y[1] += (x[1] ≥ 0 ? 1 : -1)*norm(x) 
    w = y/norm(y)
    I - 2*w*w'
end
function householderqr(A)
    m,n = size(A)
    R = copy(A)
    Q = Matrix(1.0I, m, m)
    for j = 1:n
        Qⱼ = householderreflection(R[j:end,j])
        R[j:end,:] = Qⱼ*R[j:end,:]
        Q[:,j:end] = Q[:,j:end]*Qⱼ
    end
    Q,R
end

m,n = 7,5
A = randn(m, n)
Q,R = householderqr(A)
Q*R ≈ A

true

Note because we are forming a full matrix representation of each Householder
reflection this is a slow algorithm, taking $O(n^4)$ operations. The problem sheet
will consider a better implementation that takes $O(n^3)$ operations.


**Example** We will now do an example by hand. Consider the $4 \times 3$ matrix
$$
A = \begin{bmatrix} 
1 & 2 & -1 \\ 
0 & 15 & 18 \\
-2 & -4 & -4 \\
-2 & -4 & -10
\end{bmatrix}
$$
For the first column we have
$$
Q_1 = I - {1 \over 12} \begin{bmatrix} 4 \\ 0 \\ -2 \\ -2 \end{bmatrix} \begin{bmatrix} 4 & 0 & -2 & -2 \end{bmatrix} =
{1 \over 3} \begin{bmatrix}
-1 & 0 & 2 & 2 \\
0 & 3 & 0 & 0 \\
2 & 0 & 2 & -1 \\
2 & 0 & -1 &2
\end{bmatrix}
$$
so that
$$
Q_1 A = \begin{bmatrix} -3 & -6 & -9 \\
 & 15 & 18 \\
  & 0 & 0 \\
& 0 & -6
\end{bmatrix}
$$
In this example the next column is already upper-triangular,
but because of our choice of reflection we will end up swapping the sign, that is
$$
Q_2 = \begin{bmatrix} 1 \\ & -1 \\ && 1 \\ &&& 1 \end{bmatrix}
$$
so that
$$
Q_2 Q_1 A = \begin{bmatrix} -3 & -6 & -9 \\
 & -15 & -18 \\
  & 0 & 0 \\
& 0 & -6
\end{bmatrix}
$$
The final reflection is
$$
Q_3 = \begin{bmatrix} I_{2 \times 2} \\ &  I - \begin{bmatrix} 1 \\ -1 \end{bmatrix} \begin{bmatrix} 1 & -1 \end{bmatrix} 
\end{bmatrix} = \begin{bmatrix} å1 \\ & 1 \\ & & 0 & 1 \\ & & 1 & 0 \end{bmatrix}
$$
giving us
$$
Q_3 Q_2 Q_1 A = \underbrace{\begin{bmatrix} -3 & -6 & -9 \\
 & -15 & -18 \\
  &  & -6 \\
&  & 0
\end{bmatrix}}_R
$$
That is,
$$
A = Q_1 Q_2 Q_3 R = \underbrace{{1 \over 3} \begin{bmatrix}
-1 & 0 & 2 & 2 \\
0 & 3 & 0 & 0 \\
2 & 0 & -1 & 2 \\
2 & 0 & 2 &-1
\end{bmatrix}}_Q \underbrace{\begin{bmatrix} -3 & -6 & -9 \\
 & -15 & -18 \\
  &  & -6 \\
&  & 0
\end{bmatrix}}_R = \underbrace{{1 \over 3} \begin{bmatrix}
-1 & 0 & 2  \\
0 & 3 & 0  \\
2 & 0 & -1  \\
2 & 0 & 2 
\end{bmatrix}}_Q̂  \underbrace{\begin{bmatrix} -3 & -6 & -9 \\
 & -15 & -18 \\
  &  & -6 
\end{bmatrix}}_R̂
$$



## 2. PLU Decomposition

Just as Gram–Schmidt can be reinterpreted as a reduced QR decomposition,
Gaussian elimination with pivoting can be interpreted as a PLU decomposition.


### Special "one-column" lower triangular matrices


Consider the following set of $n × n$ lower triangular
matrices which equals identity apart from one-column:
$$
{\cal L}_j := \left\{I + \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ : 𝐥_j ∈ ℝ^{n-j} \right\}
$$
where  $𝟎_j$ denotes the zero vector of length $j$. 
That is, if $L_j ∈ {\cal L}_j$ then it is equal to the identity matrix apart from in the $j$ th column:
$$
L_j = \begin{bmatrix}
        1 \\ & {⋱} \\ && 1 \\
                    && ℓ_{j+1,j} & 1 \\
                    && ⋮ && \dots \\
                    && ℓ_{n,j} & & & 1 \end{bmatrix} = 
$$

These satisify the following special properties:

**Proposition (one-column lower triangular inverse)**
If $L_j \in {\cal L}_j$ then
$$
L_j^{-1}  = I - \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ = \begin{bmatrix}
        1 \\ & ⋱ \\ && 1 \\
                    &&-ℓ_{j+1,j} & 1 \\
                    &&⋮ && \dots \\
                    &&-ℓ_{n,j} & & & 1 \end{bmatrix} ∈ {\cal L}_j.
$$


**Proposition (one-column lower triangular multiplication)**
If $L_j \in {\cal L}_j$ and $L_k \in {\cal L}_k$ for $k ≥ j$ then
$$
L_j L_k =  I + \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ +  \begin{bmatrix} 𝟎_k \\ 𝐥_k \end{bmatrix} 𝐞_k^⊤
$$


**Lemma (one-column lower triangular with pivoting)**
If $σ$ is a permutation that leaves the first $j$
rows fixed (that is, $σ_ℓ = ℓ$ for $ℓ ≤ j$) and $L_j ∈ {\cal L}_j$ then
$$
P_σ L_j=  \tilde L_j P_σ
$$
where $\tilde L_j ∈ {\cal L}_j$.

**Proof**
Write
$$
P_σ = \begin{bmatrix} I_j \\ & P_τ \end{bmatrix}
$$
where $τ$ is the permutation with Cauchy notation
$$
\begin{pmatrix}
1 & \cdots & n-j \\
σ_{j+1}-j & ⋯ & σ_n-j
\end{pmatrix}
$$
Then we have
$$
P_σ L_j = P_σ + \begin{bmatrix} 𝟎_j \\ P_τ 𝐥_j \end{bmatrix} 𝐞_j^⊤ =
\underbrace{(I +  \begin{bmatrix} 𝟎_j \\ P_τ 𝐥_j \end{bmatrix} 𝐞_j^⊤)}_{\tilde L_j} P_σ
$$
noting that $𝐞_j^⊤ P_σ = 𝐞_j^⊤$ (as $σ_j = j$). 
∎


### LU Decomposition

Before discussing pivoting, consider standard Gaussian elimation where one row-reduces
to introduce zeros column-by-column. We will mimick the computation of the QR decomposition
to view this as a _triangular triangularisation_.


Consider the matrix
$$
L_1 = \begin{bmatrix} 1 \\ -{a_{21} \over a_{11}} & 1 \\ ⋮ &&⋱ \\
                -{a_{n1} \over a_{11}}  &&& 1
                \end{bmatrix} = I - \begin{bmatrix} 0 \\ {𝐚_1[2:n] \over 𝐚_1[1]} \end{bmatrix}  𝐞_1^\top.
$$
We then have
$$
L_1 A =  \begin{bmatrix} u_{11} & u_{12} & \cdots & u_{1n} \\ 
& 𝐚_2^1 & \cdots & 𝐚_n^1   \end{bmatrix}
$$
where $𝐚_j^1 := (L_1 𝐚_j)[2:n]$ and $u_{1j} = a_{1j}$. But now consider
$$
L_2 := I - \begin{bmatrix} 0 \\ {𝐚_2^1[2:n-1] \over 𝐚_2^1[1]} \end{bmatrix}  𝐞_1^\top.
$$
Then
$$
L_2 L_1 A = \begin{bmatrix} u_{11} & u_{12} & u_{13} & \cdots & u_{1n} \\ 
    & u_{22} & u_{23} & \cdots & u_{2n} \\
&& 𝐚_3^2 & \cdots & 𝐚_n^2   \end{bmatrix}
$$
where 
$$
u_{2j} :=  (𝐚_j^1)[1] \qquad \hbox{and} \qquad 𝐚_j^2 := (L_2 𝐚_j^1)[2:n-1]
$$
Thus the first two columns are triangular. 

The inductive construction is again clear. If we define $𝐚_j^0 := 𝐚_j$ we
have the construction
$$
\begin{align*}
L_j &:= I + \begin{bmatrix} 𝟎_j \\ {𝐚_{j+1}^j[2:n-j] \over 𝐚_{j+1}^j[1]} \end{bmatrix} 𝐞_j^⊤ \\
𝐚_j^k &:= (L_k 𝐚_j^{k-1})[2:n-k+1]
 \\
u_{kj} &:= (L_k 𝐚_j^{k-1})[1]
\end{align*}
$$
Then
$$
L_{n-1} \cdots L_1 A = \underbrace{\begin{bmatrix} 
u_{11} & \cdots & u_{1n} \\ & ⋱ & ⋮\\
                                        && u_{nn}\end{bmatrix}}_U
$$
i.e.
$$
A = \underbrace{L_{n-1}^{-1} \cdots L_1^{-1}}_L U.
$$

Writing
$$
L_j = I + \begin{bmatrix} 𝟎_j \\ \ell_{j+1,j} \\ ⋮ \\ \ell_{n,j} \end{bmatrix} 𝐞_j^\top
$$
and using the properties of inversion and multiplication we therefore deduce
$$
L = \begin{bmatrix} 1 \\ -\ell_{21} & 1 \\
-\ell_{31} & -\ell_{32} & 1 \\
 ⋮ & ⋮ & ⋱ & ⋱ \\
    -\ell_{n1} & -\ell_{n2} & \cdots & -\ell_{n,n-1} & 1
    \end{bmatrix}
$$




**Example (computer)**
We will do a numerical example (by-hand is equivalent to Gaussian elimination).
The first lower triangular matrix is:

In [12]:
n = 4
A = randn(n,n)
L₁ = I -[0; A[2:end,1]/A[1,1]] * [1 zeros(1,n-1)]

4×4 Matrix{Float64}:
  1.0        -0.0  -0.0  -0.0
  1.15995     1.0  -0.0  -0.0
 -0.0724294  -0.0   1.0  -0.0
  0.400929   -0.0  -0.0   1.0

Which indeed introduces zeros in the first column:

In [13]:
A₁ = L₁*A

4×4 Matrix{Float64}:
  1.16946      -1.77156     1.27168  -0.519889
  0.0          -0.528188    1.18052  -0.312403
  0.0          -0.00625178  1.14613  -0.195066
 -5.55112e-17  -0.57422     1.64806  -0.917347

Now we make the next lower triangular operator:

In [14]:
L₂ = I - [0; 0; A₁[3:end,2]/A₁[2,2]] * [0 1 zeros(1,n-2)]

4×4 Matrix{Float64}:
  1.0  -0.0        -0.0  -0.0
 -0.0   1.0        -0.0  -0.0
 -0.0  -0.0118363   1.0  -0.0
 -0.0  -1.08715    -0.0   1.0

So that

In [15]:
A₂ = L₂*L₁*A

4×4 Matrix{Float64}:
  1.16946      -1.77156      1.27168   -0.519889
  0.0          -0.528188     1.18052   -0.312403
  0.0           0.0          1.13216   -0.191368
 -5.55112e-17  -5.55112e-17  0.364656  -0.577719

The last one is:

In [16]:
L₃ = I - [0; 0; 0; A₂[4:end,3]/A₂[3,3]] * [0 0 1 zeros(1,n-3)]

4×4 Matrix{Float64}:
  1.0  -0.0  -0.0       -0.0
 -0.0   1.0  -0.0       -0.0
 -0.0  -0.0   1.0       -0.0
 -0.0  -0.0  -0.322089   1.0

Giving us

In [17]:
U = L₃*L₂*L₁*A

4×4 Matrix{Float64}:
  1.16946      -1.77156      1.27168      -0.519889
  0.0          -0.528188     1.18052      -0.312403
  0.0           0.0          1.13216      -0.191368
 -5.55112e-17  -1.11022e-16  2.22045e-16  -0.516081

and

In [18]:
L = inv(L₁)*inv(L₂)*inv(L₃)

4×4 Matrix{Float64}:
  1.0        0.0        0.0       0.0
 -1.15995    1.0        0.0       0.0
  0.0724294  0.0118363  1.0       0.0
 -0.400929   1.08715    0.322089  1.0

Note how the entries of `L` are indeed identical to the negative
lower entries of `L₁`, `L₂` and `L₃`.

**Example (by-hand)**

Consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We have
$$
L_2 L_1 A = L_2 \begin{bmatrix}1 \\ 
                        -2 & 1 \\ -1 &  & 1 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
    = \begin{bmatrix}1 \\ & 1\\ & -{3 \over 2} & 1 
    \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\
                         & 2 & 6 \\
                         & 3 & 8 \end{bmatrix}
    = \underbrace{\begin{bmatrix} 1 & 1 & 1 \\
                         & 2 & 6 \\
                         &  & -1 \end{bmatrix}}_U
$$
We then deduce $L$ by taking the negative of the lower 
entries of $L_1,L_2$:
$$
L = \begin{bmatrix} 1 \\ 2 & 1 \\ 1 &{3 \over 2} & 1
\end{bmatrix}.
$$


### PLU Decomposition

We learned in first year linear algebra that if a diagonal entry is zero
when doing Gaussian elimnation one has to _row pivot_. For stability, 
in implementation one _always_ pivots: swap the largest in magnitude entry for the entry on the diagonal.
In terms of a decomposition, this leads to 
$$
L_{n-1} P_{n-1} \cdots P_2 L_1 P_1 A = U
$$
where $P_j$ is a permutation that leaves rows 1 through $j-1$ fixed,
and swaps row $j$ with a row $k \geq j$ whose entry is maximal in absolute value.

Thus we can deduce that 
$$
L_{n-1} P_{n-1} \cdots P_2 L_1 P_1 = \underbrace{L_{n-1} \tilde L_{n-2} \cdots  \tilde L_1}_{L^{-1}}  \underbrace{P_{n-1} \cdots P_2 P_1}_P.
$$
where the tilde denotes the combined actions of swapping the permutation and lower-triangular matrices, that is,
$$
P_{n-1}\cdots P_{j+1} L_j = \tilde L_j P_{n-1}\cdots P_{j+1}.
$$
where $\tilde L_j \in {\cal L}_j$.
The entries of $L$ are then again the negative of the entries below the diagonal of $L_{n-1}, \tilde L_{n-2}, \ldots,\tilde L_1$.


Writing
$$
\tilde L_j = I + \begin{bmatrix} 𝟎_j \\ \tilde \ell_{j+1,j} \\ ⋮ \\ \tilde \ell_{n,j} \end{bmatrix} 𝐞_j^\top
$$
and using the properties of inversion and multiplication we therefore deduce
$$
L = \begin{bmatrix} 
1 \\ 
-\tilde \ell_{21} & 1 \\
-\tilde \ell_{31} & -\tilde \ell_{32} & 1 \\
 ⋮ & ⋮ & ⋱ & ⋱ \\
 -\tilde \ell_{n-1,1} & -\tilde \ell_{n-1,2} & \cdots &  - \tilde \ell_{n-1,n-2} & 1 \\
    -\tilde \ell_{n1} & -\tilde \ell_{n2} & \cdots &  - \tilde \ell_{n,n-2} &  -\ell_{n,n-1} & 1
\end{bmatrix}
$$



**Example**

Again we consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
Even though $a_{11} = 1 \neq 0$, we still pivot: placing 
the maximum entry on the diagonal to mitigate numerical errors.
That is, we first pivot and upper triangularise the first column:
$$
 L_1 P_1 A =  L_1\begin{bmatrix} 0 & 1 \\ 1 & 0 \\ && 1 \end{bmatrix}
\begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix} = 
                     \begin{bmatrix}1 \\ -{1 \over 2} & 1 \\ -{1 \over 2} && 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                1 & 1 & 1 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We now pivot and upper triangularise the second column:
$$
  L_2 P_2 L_1 P_1 A = 
                    L_2 \begin{bmatrix}
                    1 \\ &0 & 1 \\ &1 & 0 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & -1 & -3 \\
                    0 & 2 & 5
                    \end{bmatrix}
                    = \begin{bmatrix} 1\\ & 1 \\ & {1 \over 2} & 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & -1 & -3 
                    \end{bmatrix} = 
                    \underbrace{\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & 0 & -{1 \over 2}
                    \end{bmatrix}}_U
$$
We now move $P_2$ to the right:
$$
L_2 P_2 L_1 P_1 = \underbrace{\begin{bmatrix} 1\\ -{1 \over 2} & 1 \\  -{1 \over 2}  & {1 \over 2} & 1 \end{bmatrix}}_{L_2 \tilde L_1} \underbrace{\begin{bmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \\ 1 & 0 & 0 \end{bmatrix}}_P
$$
That is
$$
L = \begin{bmatrix} 1\\ {1 \over 2} & 1 \\  {1 \over 2}  & -{1 \over 2} & 1 \end{bmatrix}
$$

We see how this example is done on a computer:

In [19]:
A = [1 1 1;
     2 4 8;
     1 4 9]
L,U,σ = lu(A) # σ is a vector encoding the permutation

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0   0.0  0.0
 0.5   1.0  0.0
 0.5  -0.5  1.0
U factor:
3×3 Matrix{Float64}:
 2.0  4.0   8.0
 0.0  2.0   5.0
 0.0  0.0  -0.5

The permutation is

In [20]:
σ

3-element Vector{Int64}:
 2
 3
 1

Thus to invert a system we can do:

In [21]:
b = randn(3)
U\(L\b[σ]) == A\b

true

Note the entries match exactly because this precisely what `\` is using.

## 3. Cholesky Decomposition

Cholesky Decomposition is a form of Gaussian elimination (without pivoting)
that exploits symmetry in the problem, resulting in a substantial speedup. 
It is only relevant for _symmetric positive definite_
matrices.

**Definition (positive definite)** A square matrix $A \in ℝ^{n \times n}$ is _positive definite_ if
for all $𝐱 \in ℝ^n$ we have
$$
𝐱^\top A 𝐱 > 0
$$

First we establish some basic properties of positive definite matrices:

**Proposition (conj. pos. def.)** If  $A \in ℝ^{n \times n}$ is positive definite and 
$V \in ℝ^{n \times n}$ is non-singular then
$$
V^\top A V
$$
is positive definite.

**Proposition (diag positivity)** If $A \in ℝ^{n \times n}$ is positive definite
then its diagonal entries are positive: $a_{kk} > 0$.


**Theorem (subslice pos. def.)** If $A \in ℝ^{n \times n}$ is positive definite
and $𝐤 \in \{1,\ldots,n\}^n$ where any integer appears only once then $A[𝐤,𝐤]$ is also
positive definite.



We leave the proofs to the problem sheets. Here is the key result:


**Theorem (Cholesky and sym. pos. def.)** A matrix $A$ is symmetric positive definite if and only if it has a Cholesky decomposition
$$
A = L L^⊤
$$
where the diagonals of $L$ are positive.

**Proof** If $A$ has a Cholesky decomposition it is symmetric ($A^⊤ = (L L^⊤)^⊤ = A$) and for $𝐱 ≠ 0$ we have
$$
𝐱^⊤ A 𝐱 = (L𝐱)^⊤ L 𝐱 = \|L𝐱\|^2 > 0
$$
where we use the fact that $L$ is non-singular.

For the other direction we will prove it by induction, with the $1 × 1$ case being trivial. 
Write
$$
A = \begin{bmatrix} α & 𝐯^\top \\
                    𝐯   & K
                    \end{bmatrix} = \underbrace{\begin{bmatrix} \sqrt{α} \\ 
                                    {𝐯 \over \sqrt{α}} & I \end{bmatrix}}_{L_1}
                                    \underbrace{\begin{bmatrix} 1  \\ & K - {𝐯 𝐯^\top \over α} \end{bmatrix}}_{A_1}
                                    \underbrace{\begin{bmatrix} \sqrt{α} & {𝐯^\top \over \sqrt{α}} \\
                                     & I \end{bmatrix}}_{L_1^\top}.
$$
Note that $K - {𝐯 𝐯^\top \over α}$ is a subslice of $A_1 = L_1^{-1} A L_1^{-⊤}$, hence by the previous propositions is
itself symmetric positive definite. Thus we can write 
$$
K - {𝐯 𝐯^\top \over α} = \tilde L \tilde L^⊤
$$
and hence $A = L L^⊤$ for
$$
L= L_1 \begin{bmatrix}1 \\ & \tilde L \end{bmatrix}.
$$
satisfies $A = L L^⊤$.
∎


Note hidden in this proof is a simple algorithm form computing the Cholesky decomposition.
We define
$$
\begin{align*}
A_1 &:= A \\
𝐯_k &:= A_k[2:n-k+1,1] \\
α_k &:= A_k[1,1] \\
A_{k+1} &:= A_k[2:n-k+1,2:n-k+1] - {𝐯_k 𝐯_k^⊤ \over α_k}.
\end{align*}
$$
Then
$$
L = \begin{bmatrix} \sqrt{α_1} \\
    {𝐯_1[1] \over \sqrt{α_1}}  &  \sqrt{α_2} \\
    {𝐯_1[2] \over \sqrt{α_1}}  & {𝐯_2[1] \over \sqrt{α_2}} &  \sqrt{α_2} \\
    ⋮ & ⋮ & ⋱ & ⋱ \\
    {𝐯_1[n-1] \over \sqrt{α_1}} &{𝐯_2[n-2] \over \sqrt{α_2}} & ⋯ & {𝐯_{n-1}[1] \over \sqrt{α_{n-1}}} & \sqrt{α_{n}}
    \end{bmatrix}
$$

This algorithm succeeds if and only if $A$ is symmetric positive definite.

**Example** Consider the matrix
$$
A_0 = A = \begin{bmatrix}
2 &1 &1 &1 \\
1 & 2 & 1 & 1 \\
1 & 1 & 2 & 1 \\
1 & 1 & 1 & 2
\end{bmatrix}
$$
Then
$$
A_1 = \begin{bmatrix}
2 &1 &1 \\
1 & 2 & 1 \\
1 & 1 & 2 
\end{bmatrix} - {1 \over 2} \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \end{bmatrix} =
{1 \over 2} \begin{bmatrix}
3 & 1 & 1 \\
1 & 3 & 1 \\
1 & 1 & 3 
\end{bmatrix}
$$
Continuing, we have 
$$
A_2 = {1 \over 2} \left( \begin{bmatrix}
3 & 1 \\ 1 & 3
\end{bmatrix} - {1 \over 3} \begin{bmatrix} 1 \\ 1  \end{bmatrix} \begin{bmatrix} 1 & 1  \end{bmatrix}
\right)
= {1 \over 3} \begin{bmatrix} 4 & 1 \\ 1 & 4 \end{bmatrix}
$$
Finally
$$
A_3 = {5 \over 4}.
$$
Thus we get
$$
L= L_1 L_2 L_3 = \begin{bmatrix} \sqrt{2} \\ {1 \over \sqrt{2}} & {\sqrt{3} \over 2} \\ 
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {2 \over \sqrt{3}} \\
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {1 \over \sqrt{12}} & {\sqrt{5} \over 2}
\end{bmatrix}
$$


# 4. Timings

The different decompositions have trade-offs between speed and stability.
First we compare the speed of the different decompositions on a symmetric positive
definite matrix, from fastest to slowest:

In [22]:
n = 100
A = Symmetric(rand(n,n)) + 100I # shift by 10 ensures positivity
@btime cholesky(A);
@btime lu(A);
@btime qr(A);

  28.524 μs (5 allocations: 78.25 KiB)
  55.173 μs (4 allocations: 79.08 KiB)
  221.079 μs (7 allocations: 134.55 KiB)


On my machine, `cholesky` is ~1.5x faster than `lu`,  
which is ~2x faster than QR. 



In terms of stability, QR computed with Householder reflections
(and Cholesky for positive definite matrices) are stable, 
whereas LU is usually unstable (unless the matrix
is diagonally dominant). PLU is a very complicated story: in theory it is unstable,
but the set of matrices for which it is unstable is extremely small, so small one does not
normally run into them.

Here is an example matrix that is in this set.

In [23]:
function badmatrix(n)
    A = Matrix(1I, n, n)
    A[:,end] .= 1
    for j = 1:n-1
        A[j+1:end,j] .= -1
    end
    A
end
A = badmatrix(5)

5×5 Matrix{Int64}:
  1   0   0   0  1
 -1   1   0   0  1
 -1  -1   1   0  1
 -1  -1  -1   1  1
 -1  -1  -1  -1  1

Note that pivoting will not occur (we do not pivot as the entries below the diagonal are the same magnitude as the diagonal), thus the PLU Decomposition is equivalent to an LU decomposition:

In [24]:
L,U = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
5×5 Matrix{Float64}:
  1.0   0.0   0.0   0.0  0.0
 -1.0   1.0   0.0   0.0  0.0
 -1.0  -1.0   1.0   0.0  0.0
 -1.0  -1.0  -1.0   1.0  0.0
 -1.0  -1.0  -1.0  -1.0  1.0
U factor:
5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0   1.0
 0.0  1.0  0.0  0.0   2.0
 0.0  0.0  1.0  0.0   4.0
 0.0  0.0  0.0  1.0   8.0
 0.0  0.0  0.0  0.0  16.0

But here we see an issue: the last column of `U` is growing exponentially fast! Thus when `n` is large
we get very large errors:

In [25]:
n = 100
b = randn(n)
A = badmatrix(n)
norm(A\b - qr(A)\b) # A \ b still uses lu

1182.3234752106287

Note `qr` is completely fine:

In [26]:
norm(qr(A)\b - qr(big.(A)) \b) # roughly machine precision

1.564181439435552377841496863828712312328754652247992940343369886191340642507706e-14

Amazingly, PLU is fine if applied to a small perturbation of `A`:

In [27]:
ε = 0.000001
Aε = A .+ ε .* randn.()
norm(Aε \ b - qr(Aε) \ b) # Now it matches!

8.78477899875885e-15

The big _open problem_ in numerical linear algebra is to prove that the set of matrices
for which PLU fails has extremely small measure.